# Multi-Agent 


In [ ]:
%pip install crewai crewai-tools longchain-ibm

In [ ]:
import os
from crewai import Crew, Task, Agent
from crewai_tools import SerperDevTool
from langchain_ibm import WatsonxLLM


## Set the API key and URL for the WatsonXLLM


In [ ]:
os.environ["WATSONX_API_KEY"] = "YOUR_API_KEY"
os.environ["WATSONX_API_KEY"] = "YOUR_API_KEY"
os.environ["WATSONX_API_KEY"] = "YOUR_API_KEY"


## LLMs

In [ ]:
parameters = {
    "deconding_method": "greedy",
    "max_new_tokens": 500, # max number of tokens to generate
    "min_new_tokens": 1, # min number of tokens to generate
    "stop_sequences": ["\n"], # stop generation at a newline
    "temperature": 0.0, # temperature for generation
    "top_k": 1, # top k candidates
    "top_p": 0.95, # top p for nucleus sampling
}

llm = WatsonxLLM(
    model="meta-llama/llama-3-8b-instruct", 
    api_key=os.environ["WATSONX_API_KEY"],
    url=os.environ["WATSONX_URL"],
    params=parameters,
    project_id=os.environ["WATSONX_PROJECT"]
)

function_calling_llm = WatsonxLLM(
    model="ibm-mistralai/merlinite-7b", 
    api_key=os.environ["WATSONX_API_KEY"],
    url=os.environ["WATSONX_URL"],
    params=parameters,
    project_id=os.environ["WATSONX_PROJECT"]
)


## Agent Tools


In [ ]:
search = SerperDevTool()

## Agents

In [ ]:
researcher = Agent(
    llm=llm,
    function_calling_llm=function_calling_llm,
    role="Senior AI Researcher",
    goal="Find promising research in the field of quantum computing.",
    backstory="You are a veteran quantum computing researcher with a background in modern physics.",
    allow_delegation=True,
    tools=[search],
    verbose=1,
)

writer = Agent(
    llm=llm,
    function_calling_llm=function_calling_llm,
    role="Senior Speech Writer",
    goal="Write engaging and witty keynote speeches from provided research.",
    backstory="You are a veteran quantum computing writer with a background in modern physics.",
    allow_delegation=True,
    tools=[search],
    verbose=1,
)


## Tasks

In [ ]:
task_1 = Task(
    description="Search the internet and find 5 examples of promising AI research.",
    expected_output="A detailed bullet point summary on each of the topics. Each bullet point should cover the topic, background and why the innovation is useful.",
    output_file="task1output.txt",
    agent=researcher,
)

task_2 = Task(
    description="Write an engaging keynote speech on quantum computing.",
    expected_output="A detailed keynote speech with an intro, body and conclusion.",
    output_file="task2output.txt",
    agent=writer,
)


## Create and run the Crew

In [ ]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[task_1, task_2],
    verbose=1,
)

print(crew.kickoff())
